<a href="https://colab.research.google.com/github/vahedshaik/Assignment4/blob/main/Data_Preparation_AutoViML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns

In [ ]:
df = sns.load_dataset('mpg')
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


In [ ]:
df.dropna(inplace = True)
df.reset_index(drop = True,inplace = True)
X = df[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin']].copy()
y = df['mpg'].copy()

In [ ]:
!pip install featuretools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 6.5 MB/s 
     |████████████████████████████████| 291 kB 49.8 MB/s 
     |████████████████████████████████| 207 kB 47.3 MB/s 
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [ ]:
import featuretools as ft
#Setting the entity base
es = ft.EntitySet(id = 'model_mpg')
es.add_dataframe(dataframe_name = 'training', dataframe = X, make_index = True, index = 'index')


/usr/local/lib/python3.7/dist-packages/woodwork/__init__.py:23: FutureWarning: Woodwork may not support Python 3.7 in next non-bugfix release.
  "Woodwork may not support Python 3.7 in next non-bugfix release.", FutureWarning
/usr/local/lib/python3.7/dist-packages/featuretools/__init__.py:67: FutureWarning: Featuretools may not support Python 3.7 in next non-bugfix release.
  FutureWarning,


Entityset: model_mpg
  DataFrames:
    training [Rows: 392, Columns: 8]
  Relationships:
    No relationships

In [ ]:
#Setting the ID based for feature engineering
es.normalize_dataframe(base_dataframe_name = 'training', new_dataframe_name = 'cylinders', index = 'cylinders')
es.normalize_dataframe(base_dataframe_name = 'training', new_dataframe_name = 'model_year', index = 'model_year')
es.normalize_dataframe(base_dataframe_name = 'training', new_dataframe_name = 'origin', index = 'origin')

Entityset: model_mpg
  DataFrames:
    training [Rows: 392, Columns: 8]
    cylinders [Rows: 5, Columns: 1]
    model_year [Rows: 13, Columns: 1]
    origin [Rows: 3, Columns: 1]
  Relationships:
    training.cylinders -> cylinders.cylinders
    training.model_year -> model_year.model_year
    training.origin -> origin.origin

In [ ]:
#Deep Feature Synthetic
feature_matrix, feature_defs = ft.dfs(entityset = es, target_dataframe_name= 'training'
,agg_primitives= ['skew', 'count', 'std', 'mean', 'median', 'mode', 'max', 'min'],
trans_primitives = ['add_numeric', 'multiply_numeric'])
feature_matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 391
Data columns (total 96 columns):
 #   Column                                    Non-Null Count  Dtype   
---  ------                                    --------------  -----   
 0   cylinders                                 392 non-null    int64   
 1   displacement                              392 non-null    float64 
 2   horsepower                                392 non-null    float64 
 3   weight                                    392 non-null    int64   
 4   acceleration                              392 non-null    float64 
 5   model_year                                392 non-null    int64   
 6   origin                                    392 non-null    category
 7   acceleration + displacement               392 non-null    float64 
 8   acceleration + horsepower                 392 non-null    float64 
 9   acceleration + weight                     392 non-null    float64 
 10  displacement + horsepower 

# **Using Auto ViML**

In [ ]:
!pip install autoviml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 8.4 MB/s 
     |████████████████████████████████| 125 kB 40.1 MB/s 
     |████████████████████████████████| 240 kB 50.9 MB/s 
     |████████████████████████████████| 255.9 MB 46 kB/s 
     |████████████████████████████████| 569 kB 54.9 MB/s 
     |████████████████████████████████| 76.6 MB 1.3 MB/s 
     |████████████████████████████████| 268 kB 59.6 MB/s 
     |████████████████████████████████| 11.2 MB 18.4 MB/s 
     |████████████████████████████████| 965 kB 57.4 MB/s 
     |████████████████████████████████| 1.6 MB 55.7 MB/s 
     |████████████████████████████████| 121 kB 58.5 MB/s 
     |████████████████████████████████| 82 kB 894 kB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234927 sha256=4cfc651679604c4fcf915d904ba256eed57af54d2ca96b004995197c8b90f1a1
  Stored in directory: /root/.cache/pip/wheels/f3/e3/f2/1d

In [ ]:
feature_matrix['mpg'] = y

In [ ]:
features


['cylinders.STD(training.acceleration)',
 'weight',
 'model_year',
 'model_year.STD(training.weight)',
 'model_year.MAX(training.acceleration)',
 'horsepower * weight',
 'origin.COUNT(training)',
 'acceleration * weight',
 'model_year.STD(training.acceleration)',
 'model_year.MEAN(training.horsepower)',
 'model_year.COUNT(training)',
 'origin.MAX(training.acceleration)',
 'cylinders.COUNT(training)',
 'cylinders.MODE(training.origin)',
 'cylinders',
 'model_year.MODE(training.origin)',
 'model_year.SKEW(training.weight)',
 'origin.MIN(training.weight)',
 'origin.MIN(training.horsepower)']

In [ ]:
model

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=8, gpu_id=0, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.05150789303581249, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=6, max_leaves=0,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=209, n_jobs=-1, nthread=-1, num_parallel_tree=1,
             predictor='cpu_predictor', random_state=1, reg_alpha=0.5, ...)

**Using Featurewiz**

In [ ]:
!pip install featurewiz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 111 kB 7.2 MB/s 
     |████████████████████████████████| 3.2 MB 11.2 MB/s 
     |████████████████████████████████| 2.0 MB 52.5 MB/s 
     |████████████████████████████████| 26.7 MB 1.5 MB/s 
     |████████████████████████████████| 96 kB 4.1 MB/s 
     |████████████████████████████████| 72 kB 727 kB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 6.0.1
    Uninstalling pyarrow-6.0.1:
      Successfully uninstalled pyarrow-6.0.1
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm

In [ ]:
features

(['cylinders.STD(training.acceleration)',
  'model_year.MEAN(training.weight)',
  'horsepower * weight',
  'model_year.MEDIAN(training.acceleration)',
  'model_year.MAX(training.acceleration)',
  'origin.MAX(training.horsepower)',
  'model_year.STD(training.acceleration)',
  'model_year.SKEW(training.weight)',
  'cylinders.SKEW(training.weight)',
  'cylinders.MODE(training.origin)',
  'origin'],
        cylinders.STD(training.acceleration)  model_year.MEAN(training.weight)  \
 index                                                                           
 0                                      2.22                          3,372.79   
 1                                      2.22                          3,372.79   
 2                                      2.22                          3,372.79   
 3                                      2.22                          3,372.79   
 4                                      2.22                          3,372.79   
 ...                       